<a href="https://colab.research.google.com/github/esha345/GAN/blob/main/AnimeGeneratorDCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"eshaghorpade","key":"6c1d020dff74ea3105405b5df3155a93"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change the permission
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d splcher/animefacedataset

animefacedataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
file_name = "animefacedataset.zip"
with ZipFile(file_name,'r') as z:
  z.extractall()
  print('Done')

Done


In [ ]:
!nvidia-smi

Thu Feb 25 12:58:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D , Conv2DTranspose
from sklearn.utils import shuffle
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
from IPython import display
import os
from os import listdir
from os.path import splitext
import cv2
import random
import shutil
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
height = 64
width = 64
channels = 3

In [ ]:
images_path = glob("images/*")
print("Dataset Size: ",len(images_path))

Dataset Size:  63565


In [ ]:
#resizing and normalizing the images
def load_image(image_path):
  image=tf.io.read_file(image_path)
  image=tf.io.decode_jpeg(image)
  image=tf.image.resize(image,(height,width))
  image=tf.cast(image,tf.float32)
  image = (image-127.5)/127.5
  return image
#creating batches
batch_size = 128
def tf_dataset(images_path, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(images_path).shuffle(1000)
  dataset = dataset.map(load_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
dataset = tf_dataset(images_path,batch_size)
for x in dataset:
  print(x.shape)

(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 3)
(128, 64, 64, 

In [ ]:
w_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [ ]:
def deconv_block(inputs, num_filters, kernel_size, strides,bn=True):
  x = Conv2DTranspose(
      filters = num_filters,
      kernel_size = kernel_size,
      kernel_initializer = w_init,
      padding = 'same',
      strides= strides,
      use_bias = False
  )(inputs)
  if bn:
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU(alpha=0.2)(x)
  return x

def conv_block(inputs, num_filters, kernel_size, padding='same',strides=2,activation=True):
  x = Conv2D(
      filters = num_filters,
      kernel_size = kernel_size,
      kernel_initializer = w_init,
      padding = padding,
      strides = strides ,       
    )(inputs)
  if activation:
      x = keras.layers.LeakyReLU(alpha=0.2)(x) 
      x = keras.layers.Dropout(0.3)(x)
  return x
   

In [ ]:
#building generator
#(4,4,512)
#(8,8,256)
#(16,16,128)
#(32,32,64)
#(64,64,32)
latent_dim = 128
def build_generator(latent_dim):
  f = [2**i for i in range(5)][::-1]
  filters = 32
  output_strides = 16
  h_output = height // output_strides
  w_output = width // output_strides
  noise = keras.Input(shape=(latent_dim,), name="gen_input_noise")
  x = keras.layers.Dense(f[0]*filters*h_output*w_output,use_bias=False)(noise)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.LeakyReLU(alpha=0.2)(x)
  x = keras.layers.Reshape((h_output,w_output,f[0]*filters))(x)
  #creating 5 deconv blocks
  for i in range(1,5):
    x = deconv_block(x,num_filters=f[i]*filters,kernel_size=5,strides=2,bn=True)
  x = conv_block(x,num_filters=3, kernel_size=5,strides=1,activation=False)
  fake_output = keras.layers.Activation("tanh")(x)
  return keras.Model(noise, fake_output, name="generator")

In [ ]:
#building discriminator
def build_discriminator():
  f = [2**i for i in range(4)][::-1]
  filters = 64
  output_strides = 16
  h_output = height // output_strides
  w_output = width // output_strides
  image_input = keras.Input(shape=(height,width,channels), name="images")
  x = image_input
  #creating 4 conv blocks
  for i in range(0,4):
    x = conv_block(x,num_filters=f[i]*filters,kernel_size=5,strides=2)
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(1)(x)
  x = keras.layers.Activation('sigmoid')(x)
  return keras.Model(image_input, x, name="discrimintor")

In [ ]:
d_model = build_discriminator()
d_model.summary()

Model: "discrimintor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
images (InputLayer)          [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 512)       38912     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 512)       0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 512)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       3277056   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 256)      

In [ ]:
g_model = build_generator(latent_dim)
g_model.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gen_input_noise (InputLayer) [(None, 128)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              1048576   
_________________________________________________________________
batch_normalization (BatchNo (None, 8192)              32768     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 256)         3276800   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 256)         10

In [ ]:
#creating a complete GAN model
class GAN(keras.Model):
  def __init__(self,discriminator,generator,latent_dim):
    super(GAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim
  def compile(self, d_optimizer, g_optimizer, loss_fn):
    super(GAN, self).compile()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer
    self.loss_fn = loss_fn
  def train_step(self ,real_images):
    batch_size = tf.shape(real_images)[0]
    #training the discriminator
    for _ in range(2):
      #fake images
      random_latent_vectors = tf.random.normal(shape = (batch_size,self.latent_dim))
      generated_images = self.generator(random_latent_vectors)
      generated_labels = tf.zeros((batch_size,1))
      with tf.GradientTape() as tape:
        predictions = self.discriminator(generated_images)
        d1_loss = self.loss_fn(generated_labels, predictions)
      grads = tape.gradient(d1_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads,self.discriminator.trainable_weights))
      #real images
      labels = tf.ones((batch_size,1))
      with tf.GradientTape() as tape:
        predictions = self.discriminator(real_images)
        d2_loss = self.loss_fn(labels, predictions)
      grads = tape.gradient(d2_loss, self.discriminator.trainable_weights)
      self.d_optimizer.apply_gradients(zip(grads,self.discriminator.trainable_weights))
    #training the generator
    random_latent_vectors = tf.random.normal(shape = (batch_size,self.latent_dim))
    misleading_labels = tf.ones((batch_size,1))
    with tf.GradientTape() as tape:
      predictions = self.discriminator(self.generator(random_latent_vectors))
      g_loss = self.loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, self.generator.trainable_weights)
    self.g_optimizer.apply_gradients(zip(grads,self.generator.trainable_weights))
    return {"d1_loss": d1_loss, "d2_loss": d2_loss, "g_loss": g_loss}
       
  

In [ ]:
def save_plot(examples,epoch,n):
  examples = (examples+1)/2.0
  for i in range(n * n):
    plt.subplot(n,n,i+1)
    plt.axis("off")
    plt.imshow(examples[i])
  filename = f"sample_data/fake_images_epoch_{epoch+1}.jpeg"
  plt.savefig(filename)
  plt.close()


In [ ]:
if __name__=="__main__":
  num_epochs = 30
  gan = GAN(d_model, g_model, latent_dim)
  bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
  d_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  g_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
  gan.compile(d_optimizer, g_optimizer, bce_loss_fn)
  images_dataset = tf_dataset(images_path,batch_size)
  for epoch in range(num_epochs):
    gan.fit(images_dataset, epochs=1)
    g_model.save("saved_model/g_model.h5")
    d_model.save("saved_model/d_model.h5")
    n_samples = 25
    noise = np.random.normal(size = (n_samples, latent_dim))
    examples = g_model.predict(noise)
    save_plot(examples, epoch, int(np.sqrt(n_samples)))

 76/496 [===>..........................] - ETA: 21:50 - d1_loss: 0.2656 - d2_loss: 0.2198 - g_loss: 3.2750